# Merging Agricultural and Vehicle Data to AQI Dataset

In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [289]:
#read all datasets for merging
vehicle = pd.read_csv('cleaned_vehicle_registration.csv')
demo = pd.read_csv('combined demographic.csv')
crops = pd.read_csv('Cropland by municipality_cleaned.csv')
pigs = pd.read_csv('Pigs by municipality_cleaned.csv')
cattle = pd.read_csv('Cattle and calves by municipality_cleaned.csv')

In [291]:
#Read data to ensure same dtype and columns names
vehicle.dtypes

Municipality          object
Year                   int64
Vehicle_Type          object
Registration_Count     int64
dtype: object

In [293]:
comb_demo.head()

,Unique CSD,Unique Year,Combined Key,Adjusted Air Quality Health Index,Both Sexes Life Expectancy (Years),Population
0,Red Deer,2011,Red Deer2011,2.74906,79.96,75553
1,Red Deer,2012,Red Deer2012,2.43250,80.00,77674
2,Red Deer,2013,Red Deer2013,2.54466,80.17,79180
3,Red Deer,2014,Red Deer2014,2.23417,80.20,80290
4,Red Deer,2015,Red Deer2015,2.26231,80.37,80049


In [295]:
crops = crops.drop(columns=['CSDUID','IndicatorSummaryDescription'])

In [296]:
pigs = pigs.drop(columns=['CSDUID','IndicatorSummaryDescription'])

In [299]:
cattle = cattle.drop(columns=['CSDUID', 'IndicatorSummaryDescription'])

In [301]:
pigs.dtypes

CSD        object
Period      int64
Count     float64
dtype: object

In [303]:
cattle.head()

,CSD,Period,Count
0,Starland County,2001,0.0
1,Starland County,2006,34341.0
2,Starland County,2011,35178.0
3,Starland County,2016,54027.0
4,Starland County,2021,33028.0


In [305]:
crops.tail()

,CSD,Period,Crop Type,Count
1701,Two Hills County No. 21,2021,Other,62591.0
1702,Yellowhead County,2006,Other,108480.0
1703,Yellowhead County,2011,Other,95569.0
1704,Yellowhead County,2016,Other,84238.0
1705,Yellowhead County,2021,Other,80023.0


In [307]:
veh_column_map = {'Municipality':'CSD','Year':'Period','Vehicle_Type':'Vehicle_Type','Registration_Count':'Vehicle_Registrations'}

In [309]:
vehicle.rename(columns=veh_column_map, inplace=True)
vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   CSD                    480 non-null    object
 1   Period                 480 non-null    int64 
 2   Vehicle_Type           480 non-null    object
 3   Vehicle_Registrations  480 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [311]:
demo = demo.drop(columns=['Combined Key'])
demo.head()

,Unique CSD,Unique Year,Adjusted Air Quality Health Index,Both Sexes Life Expectancy (Years),Population
0,Red Deer,2011,2.74906,79.96,75553
1,Red Deer,2012,2.43250,80.00,77674
2,Red Deer,2013,2.54466,80.17,79180
3,Red Deer,2014,2.23417,80.20,80290
4,Red Deer,2015,2.26231,80.37,80049


In [313]:
demo_column_map = {'Unique CSD': 'CSD', 'Unique Year': 'Period'}

In [315]:
demo.rename(columns=demo_column_map, inplace=True)
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   CSD                                 284 non-null    object 
 1   Period                              284 non-null    int64  
 2   Adjusted Air Quality Health Index   284 non-null    float64
 3   Both Sexes Life Expectancy (Years)  267 non-null    float64
 4   Population                          284 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 11.2+ KB


In [317]:
crops.rename(columns={'Crop Type':'Crop_Type','Count': 'Crop_Count'}, inplace=True)
crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706 entries, 0 to 1705
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CSD         1706 non-null   object 
 1   Period      1706 non-null   int64  
 2   Crop_Type   1706 non-null   object 
 3   Crop_Count  1706 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 53.4+ KB


In [319]:
pigs.rename(columns={'Count': 'Pig_Count'}, inplace=True)
pigs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CSD        354 non-null    object 
 1   Period     354 non-null    int64  
 2   Pig_Count  354 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 8.4+ KB


In [321]:
cattle.rename(columns={'Count': 'Cattle_and_Calves_Count'}, inplace=True)
cattle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CSD                      352 non-null    object 
 1   Period                   352 non-null    int64  
 2   Cattle_and_Calves_Count  352 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 8.4+ KB


In [329]:
# merge time
merge1 = pd.merge(demo, vehicle, on=['CSD', 'Period'], how='outer')
merge2 = pd.merge(merge1, crops, on=['CSD', 'Period'], how='outer')
merge3 = pd.merge(merge2, pigs, on=['CSD', 'Period'], how='outer')
merge4 = pd.merge(merge3, cattle, on=['CSD', 'Period'], how='outer')
merge4

,CSD,Period,Adjusted Air Quality Health Index,Both Sexes Life Expectancy (Years),Population,Vehicle_Type,Vehicle_Registrations,Crop_Type,Crop_Count,Pig_Count,Cattle_and_Calves_Count
0,Airdrie,2005,NaN,NaN,NaN,Total Motorized Vehicles,24609.0,NaN,NaN,NaN,NaN
1,Airdrie,2006,NaN,NaN,NaN,Total Motorized Vehicles,27288.0,NaN,NaN,NaN,NaN
2,Airdrie,2007,NaN,NaN,NaN,Total Motorized Vehicles,30095.0,NaN,NaN,NaN,NaN
3,Airdrie,2008,NaN,NaN,NaN,Total Motorized Vehicles,32293.0,NaN,NaN,NaN,NaN
4,Airdrie,2009,NaN,NaN,NaN,Total Motorized Vehicles,36368.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2252,Yellowhead County,2021,2.11912,79.25,6835.0,NaN,NaN,Durum Wheat,0.0,0.0,60209.0
2253,Yellowhead County,2021,2.11912,79.25,6835.0,NaN,NaN,Non-durum Wheat,0.0,0.0,60209.0
2254,Yellowhead County,2021,2.11912,79.25,6835.0,NaN,NaN,Oats,15741.0,0.0,60209.0
2255,Yellowhead County,2021,2.11912,79.25,6835.0,NaN,NaN,Other,80023.0,0.0,60209.0


In [331]:
merge4.dtypes

CSD                                    object
Period                                  int64
Adjusted Air Quality Health Index     float64
Both Sexes Life Expectancy (Years)    float64
Population                            float64
Vehicle_Type                           object
Vehicle_Registrations                 float64
Crop_Type                              object
Crop_Count                            float64
Pig_Count                             float64
Cattle_and_Calves_Count               float64
dtype: object

In [343]:
merge4[190:210]

,CSD,Period,Adjusted Air Quality Health Index,Both Sexes Life Expectancy (Years),Population,Vehicle_Type,Vehicle_Registrations,Crop_Type,Crop_Count,Pig_Count,Cattle_and_Calves_Count
190,Bonnyville No. 87,2006,NaN,NaN,NaN,Total Motorized Vehicles,2513.0,Durum Wheat,0.0,1162.0,94119.0
191,Bonnyville No. 87,2006,NaN,NaN,NaN,Total Motorized Vehicles,2513.0,Non-durum Wheat,8921.0,1162.0,94119.0
192,Bonnyville No. 87,2006,NaN,NaN,NaN,Total Motorized Vehicles,2513.0,Oats,25376.0,1162.0,94119.0
193,Bonnyville No. 87,2006,NaN,NaN,NaN,Total Motorized Vehicles,2513.0,Other,156417.0,1162.0,94119.0
194,Bonnyville No. 87,2007,NaN,NaN,NaN,Total Motorized Vehicles,2696.0,NaN,NaN,NaN,NaN
195,Bonnyville No. 87,2008,NaN,NaN,NaN,Total Motorized Vehicles,2830.0,NaN,NaN,NaN,NaN
196,Bonnyville No. 87,2009,NaN,NaN,NaN,Total Motorized Vehicles,3010.0,NaN,NaN,NaN,NaN
197,Bonnyville No. 87,2010,NaN,NaN,NaN,Total Motorized Vehicles,3010.0,NaN,NaN,NaN,NaN
198,Bonnyville No. 87,2011,NaN,NaN,NaN,Total Motorized Vehicles,3168.0,Barley,26761.0,113.0,63418.0
199,Bonnyville No. 87,2011,NaN,NaN,NaN,Total Motorized Vehicles,3168.0,Canola,42233.0,113.0,63418.0


,CSD,Period,Adjusted Air Quality Health Index,Both Sexes Life Expectancy (Years),Population
130,Strathcona County,2015,2.30421,81.93,74131
131,Strathcona County,2016,2.13355,82.08,74185
132,Strathcona County,2017,2.38307,82.16,73842
133,Strathcona County,2018,2.57518,82.24,73301
134,Strathcona County,2019,2.34714,82.42,72911
135,Strathcona County,2020,2.26426,82.38,72206
136,Strathcona County,2021,2.35545,82.28,71127
137,Strathcona County,2022,2.33306,82.17,71041
138,Sturgeon County,2016,1.94946,79.17,15819
139,Sturgeon County,2017,2.26669,79.33,15616
